In [3]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import copy
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from scipy import stats
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
res = pd.DataFrame(0,columns=["spnsi","speci","GM","f1","AUC"],index=dataList)

In [1]:
dataList = ["MM/MM-BP.csv","MM/MM-CC.csv","MM/MM-MF.csv","MM/MM-BPCC.csv",
            "MM/MM-BPMF.csv","MM/MM-CCMF.csv","MM/MM-BPCCMF.csv",
           "CE/CE-BP.csv","CE/CE-CC.csv","CE/CE-MF.csv","CE/CE-BPCC.csv",
            "CE/CE-BPMF.csv","CE/CE-CCMF.csv","CE/CE-BPCCMF.csv",
           "DM/DM-BP.csv","DM/DM-CC.csv","DM/DM-MF.csv","DM/DM-BPCC.csv",
            "DM/DM-BPMF.csv","DM/DM-CCMF.csv","DM/DM-BPCCMF.csv",
           "SC/SC-BP.csv","SC/SC-CC.csv","SC/SC-MF.csv","SC/SC-BPCC.csv",
            "SC/SC-BPMF.csv","SC/SC-CCMF.csv","SC/SC-BPCCMF.csv"]
dataList = ["SportsTweetst.csv","SportsTweetsc.csv","NYDailyheadings.csv","Stumbleupon.csv","Cities.csv"]
dataName = "SportsTweetsc.csv"

In [6]:
#加载数据 预处理
data = pd.read_csv("../data/real_world_datasets/Datasets/"+dataName,index_col=None)
if(dataName == "Cities.csv"):
    data["longevity influence"] = data["longevity influence"].apply(lambda x : 0 if x < 2e+09 else 1)
label = data["longevity influence"]
#删除无用列
data = data.drop(["longevity influence"],axis=1)

#初始化基因本体的DAG
DAG = pd.read_csv("../data/real_world_datasets/Hierarchy/"+dataName,index_col=0)

#过滤低纬度特征
for c in data.columns:
    data[c] = data[c].astype('int')
    if(data[c].sum() < 50):
        data.drop(c,axis=1,inplace=True)
DAG = DAG.loc[DAG.columns.intersection(data.columns),DAG.columns.intersection(data.columns)]
print(data.shape)
#计算相关性
data["longevity influence"] = label
Relevance = pd.Series(0.0,index=DAG.columns)
for name in DAG.columns:
    temp = data[data[name] == 1]
    prob = temp['longevity influence'].mean()
    Relevance[name] = (prob-0.5)*(prob-0.5)+(0.5-prob)*(0.5-prob)
data = data.drop(["longevity influence"],axis=1)

#计算所有节点的祖先节点
AncDict = {}
relativeList = []
for name in DAG.columns:
    #print(name)
    global relativeList
    relativeList = []
    getAncestors(name)
    AncDict[name] = relativeList

(1179, 109)


C:\Users\Li\AppData\Local\Temp\ipykernel_5004\3754288091.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data["longevity influence"] = label


In [4]:
#函数
def GM_score(x,y): #计算GM 传入numpy
    cm1 = confusion_matrix(x,y)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    return sensitivity1,specificity1

def getPathsToRoot(path,name):#传入list 方便递归
    global pathList
    flag = 0
    path.append(name)
    for anc in DAG.columns:
        if(DAG.loc[anc,name] == 1):
            getPathsToRoot(copy.copy(path),anc)
            flag = 1
    if(flag == 0):
        pathList.append(path)
        
def RPV(x):
    dropList = []
    for name in DAG.columns:
        if(x[name] == 1):
            #遍历当前节点的所有祖先节点
            for anc in AncDict[name]:
                #保留相关性最高特征，其它特征加入dropSet
                if(Relevance[anc] < Relevance[name]):
                    dropList.append(anc)
        else:
            dropList.append(name)
    return list(set(DAG.columns)-set(dropList))

def myknn(train,label,x_test):
    predictList = []
    for j in range(x_test.shape[0]):
        test = x_test.iloc[j,:]
        maxJcd = 0
        prediction = 0
        for i in range(train.shape[0]):
            temp = jaccard_score(train.iloc[i,:],test)
            if(temp > maxJcd):
                prediction = label[i]
                maxJcd = temp
        predictList.append(prediction)
    return predictList

def getAncestors(name):
    global relativeList
    for relative in DAG.columns:
        if(DAG.loc[relative,name] == 1):
            relativeList.append(relative)
            getAncestors(relative)

In [7]:
#执行
sensitivity = []
specificity = []
F1 = []
AUC = []
numList = []

#10折交叉验证
kf = KFold(n_splits=5,shuffle=True)
for train_index ,test_index in kf.split(data):
    trainData = data.iloc[train_index,:]
    testData = data.iloc[test_index,:]
    Y_train = label.values[train_index] #用于训练模型
    Y_test = label.values[test_index]#用于交叉验证
    predicetList = []
    
    
    for i in range(testData.shape[0]):  #对每一个样本进行特征选择和预测
        
        selectFeatures = RPV(testData.iloc[i,:]) #MR特征选择
        
        print(len(selectFeatures))
        if(len(selectFeatures) == 0):
            predicetList.append(pd.Series(Y_train).mode()[0])
            continue
            #selectFeatures = DAG.columns
        else:
            numList.append(len(selectFeatures))
        X_train = trainData.loc[:,selectFeatures]
        X_test = testData.loc[:,selectFeatures].iloc[i,:]
        from sklearn.svm import SVC
        clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
        clf.fit(X_train, Y_train)
        predicetList.append(clf.fit(X_train, Y_train).predict(X_test.values.reshape(1,len(selectFeatures)))[0])
        
#         gnb = GaussianNB()
#         predicetList.append(gnb.fit(X_train, Y_train).predict(X_test.values.reshape(1,len(selectFeatures)))[0])#把当前样本预测结果加入List
        #knn=KNeighborsClassifier(n_neighbors=1)
        #knn.fit(X_train,Y_train)
        #predicetList.append(knn.predict(X_test.values.reshape(1,len(selectFeatures)))[0])
    print(predicetList)
    print(Y_test)
    sensi,speci= GM_score(np.array(predicetList),Y_test)
    sensitivity.append(sensi)
    specificity.append(speci)
    F1.append(f1_score(np.array(predicetList),Y_test))
    try:
        AUC.append(roc_auc_score(np.array(predicetList),Y_test))
    except ValueError:
        pass

#RPV
a = np.nanmean(sensitivity)
b = np.nanmean(specificity)
print(a)
print(b)
print("GM")
print(round(math.sqrt(a*b)*100,1))
print("f1")
print(round(np.nanmean(F1)*100,1))
print("AUC")
print(round(np.nanmean(AUC)*100,1))
print("选择特征百分比")
print(round(np.nanmean(numList)/DAG.shape[0]*100,1))
res.loc[dataName,"spnsi"] = round(a*100,1)
res.loc[dataName,"speci"] = round(b*100,1)
res.loc[dataName,"GM"] = round(math.sqrt(a*b)*100,1)
res.loc[dataName,"f1"] = round(np.nanmean(F1)*100,1)
res.loc[dataName,"AUC"] = round(np.nanmean(AUC)*100,1)

10
0
0
22
9
22


c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with f

21
0
0
6
38


c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


3
24
11


...

c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


0
22
0
0
0
30
0
0
4


c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


22
11
0
23
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [11]:
DAG

,Lloyd_Webber_Andrew,L,People,Arts,Directing,Theatre,Performing_Arts,Articles,News_and_Media,Networks,...,LehrerTom,Nanopunk,Miami,Consumer_Electronics,Thai,Prime_Ministers,Books,x86,Buses,Weight_Loss
Lloyd_Webber_Andrew,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
L,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
People,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Arts,0,1,1,0,1,1,1,1,1,1,...,1,1,0,0,0,0,0,0,0,0
Directing,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Prime_Ministers,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Books,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
x86,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Buses,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
